<a href="https://colab.research.google.com/github/FrancisLa/ACFAS/blob/main/ACFAS_consolidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Consolidation of ACFAS dataframe
This is where we explain what this code do.

#Initialization

##Import standard libraries

In [ ]:
import os
import sys
import subprocess
from datetime import datetime
import numpy as np
import pickle
import re
import pandas as pd
import importlib
import pkg_resources
import types

##Import non standard libraries

In [ ]:
def import_or_install(package, version=None):
    try:
        if version is None:
            importlib.import_module(package)
        else:
            dist = pkg_resources.get_distribution(package)
            assert dist.version == version, f"{package}=={version} not found"
    except (ImportError, pkg_resources.DistributionNotFound, AssertionError):
        if version is None:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}"])
    finally:
        globals()[package] = importlib.import_module(package)

import_or_install('platform')
import_or_install('socket')
import_or_install('uuid')
import_or_install('psutil')
import_or_install('GPUtil')

##Project name, path and authorization


This code uses Google drive to access the data. The project_name will be used as the main directory in Google drive (inside Colab data). It can be run locally (hosted = False) or through Colab hosting (hosted = True).

In [ ]:
project_name = 'ACFAS'
operation_name = 'consolidation'
hosted = True

###Colab hosting

In [ ]:
if hosted:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  project_dir = '/content/gdrive/My Drive/Colab Data/' + project_name
  tools_dir = '/content/gdrive/My Drive/Colab Data/Tools'
  operation_dir = tools_dir = '/content/gdrive/My Drive/Colab Data/'+ project_name+'/'+ operation_name
  os.makedirs(operation_dir, exist_ok=True)

Mounted at /content/gdrive


###Local hosting
Local runtime need some preparation (https://research.google.com/colaboratory/local-runtimes.html)

It might be safer to work with python 3.7.9 (https://python.org/downloads/release/python-379/)

You may need to run this command line :

jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=9090 --no-browser

You may use google drive by installing gdrive locally.

In [ ]:
if not hosted:
  try:
    os.chdir(os.path.join('G:',os.sep,'Mon disque'))
  except:
    os.chdir(os.path.join('G:',os.sep,'My Drive'))
  project_dir = os.path.join('Colab Data',project_name)
  operation_dir = os.path.join('Colab Data',project_name,operation_name)
  tools_dir = os.path.join('Colab Data','Tools')
  os.makedirs(operation_dir, exist_ok=True)

## Install requirements

If you have a text file named *project_name_operation_name_requirements.txt* in *operation_dir*, with package and version on each line ('re==', or 'pandas==2.0.3'), this code will automatically show them.

In [ ]:
if os.path.isfile(os.path.join(operation_dir, project_name+'_'+operation_name+'_requirements.txt')):
  with open(os.path.join(operation_dir, project_name+'_'+operation_name+'_requirements.txt'), 'r') as file_:
    for line in file_:
      package, version = line.strip().split('==')
      if version != '':
        print(f"Package {package}, version {version}")
      else:
        #print(f"Package {package}")
        pass

#Import data

Initialisation des valeurs constantes qui contiennent les chemins des documents sources et document cible.
Le document source est une table à deux dimentions dont les noms des colonnes sont dans la 1e ligne.

In [ ]:
# Constantes de programme
PATH_SOURCE = '' # entrer le chemin du fichier à ingérer
BALISEATOMISATION = '{Atomisation incomplète}' # spécifier la balise d'incomplétude pour cette ingestion. Lorsque ce texte se retrouve dans un champs ingéré,
                    #   il signifi que l'atomisation est incomplète et les valeurs dans la cellule ont besoin d'être extraites à une étape ultérieure.
SEPARATEUR = "$" # séparateur sert à marquer les différents champs concatenés à l'ingestion.
CHARGEMENT = '3'

In [ ]:
# Fonction pour générer un hash MD5
import hashlib
def generate_md5_hash(value):
    return hashlib.md5(value.encode('utf-8')).hexdigest()

In [ ]:
## Bloc 2 - table Programme

def process_file(file_path, df_cible):
    # Check the file extension and read the file accordingly
    if file_path.endswith('.csv'):
        df_source = pd.read_csv(file_path)
        print("le fichier csv",file_path," sera traité")
    elif file_path.endswith('.xlsx'):
        df_source = pd.read_excel(file_path)
        print("le fichier xlsx", file_path," sera traité")
    else:
        raise ValueError("Unsupported file format")

    #display(df_source)
    #charger le document cible qui contient les noms de toutes les colonnes
    # voir Variables&Lexique.xlsx


    # transfert des énoncés sources dans leurs tables respectives
    # décommenter la ligne puis ajouté le nom de la source après =
    #Inscrit la date de l'ingestion  dans le dataset
    x=datetime.now()
    df_cible['Date_Ingestion']=x

    # Transfert de la date du congrès
    df_cible['Temporalite_Date'] = df_source['jour']
    # df_cible['_Temporalité_Clé]
    # df_cible['_Temporalité_Id]
    df_cible['Temporalite_EnonceSource']=df_source['notice_complete']
    df_cible['Temporalite_Analyste']='Table des programme compilée par Luc Gauvreau et Julien Vallières Gingras'
    df_cible['Temporalite_ReferenceBiblio']="{1933-1962 : Programme du [no congrès] congrès, [année du congrès]/1963-2001 : Programme général : [no congrès] congrès annuel, [année du congrès]}"


    # # transfert des élément de l'évènement
    # df_cible['Evenement_CleEvenement']
    # df_cible['Evenement_IdEvenement']
    df_cible['Evenement_TitreEvenement']=df_source['titre']
    # df_cible['Evenement_CleTemporaliteDebEvenement']
    # df_cible['Evenement_CleTemporaliteFinEvenement']
    # df_cible['Evenement_SeqDistinctiveEvenement']
    df_cible['Evenement_DtHrChargement1']=CHARGEMENT
    # df_cible['Evenement_CleEvenementSource']
    df_cible['Evenement_TitreLongEvenement']=df_source['titre']
    df_cible['Evenement_ResumeEvenement']=df_source['resume']
    df_cible['Evenement_DescEvenement']=df_source['description']
    df_cible['Evenement_Analyste']='Table des programme compilée par Luc Gauvreau et Julien Vallières Gingras'
    df_cible['Evenement_EnonceSource']=df_source['notice_complete']
    df_cible['Evenement_ReferenceBiblio']="{1933-1962 : Programme du [no congrès] congrès, [année du congrès]/1963-2001 : Programme général : [no congrès] congrès annuel, [année du congrès]}"


    #transfert des valeurs dans la table Peronnes
    df_cible["Personne_AliasPersonne"] = df_source['auteur']
    df_cible['Personne_AutoratEnonce'] = df_source['auteur']
    # df_cible['Personne_ClePersonne']
    # df_cible['Personne_IdPersonne']
    df_cible['Personne_PrenomPersonne'] = BALISEATOMISATION+df_source['auteur']
    df_cible['Personne_DeuxiemePrenomPersonne']=BALISEATOMISATION+df_source['auteur']
    df_cible['Personne_NomFamillePersonne']=BALISEATOMISATION+df_source['auteur']
    df_cible['Personne_TitrePersonne']=BALISEATOMISATION+df_source['auteur']
    # df_cible['Personne_AnneeNaissancePersonne']
    # df_cible['Personne_AnneeDecesPersonne']
    # df_cible['Personne_SeqDistinctivePersonne']
    df_cible['Personne_DtHrChargement1']=CHARGEMENT
    df_cible['Personne_PersonneRoles']= BALISEATOMISATION+df_source['personne_roles']
    # df_cible['Personne_CleEvenementSource')
    df_cible['Personne_EnonceSource'] = df_source['notice_complete']
    df_cible['Personne_Analyste']='Table des programme compilée par Luc Gauvreau et Julien Vallières Gingras'
    df_cible['Personne_ReferenceBiblio']="{1933-1962 : Programme du [no congrès] congrès, [année du congrès]/1963-2001 : Programme général : [no congrès] congrès annuel, [année du congrès]}"


    #transfert vers la table Collectivité
    # df_cible['Collectivite_CleCollectivite']
    # df_cible['Collectivite_IdCollectivite']
    df_cible['Collectivite_NomCollectivite'] = BALISEATOMISATION+df_source['auteur']
    # df_cible['Collectivite_DescCollectivite']
    df_cible['Collectivite_AliasCollectivite'] = BALISEATOMISATION+df_source['auteur']
    df_cible['Collectivite_AutoratEnonce'] = df_source['auteur']
    # df_cible['Collectivite_AnneeFondationCollectivite']
    # df_cible['Collectivite_AnneeDissolutionCollectivite']
    # df_cible['Collectivite_SeqDistinctiveCollectivite0']
    df_cible['Collectivite_DtHrChargement1']=CHARGEMENT
    # df_cible['Collectivite_CleEvenementSource']
    df_cible['Collectivite_Analyste'] = "compilé par Julien Vallières et Luc Gauvreau"
    df_cible['Collectivite_ReferenceBiblio'] = "{1933-1962 : Programme du [no congrès] congrès, [année du congrès]/1963-2001 : Programme général : [no congrès] congrès annuel, [année du congrès]}"
    df_cible['Collectivite_EnonceSource'] = df_source['notice_complete']


    #Transfert vers la table Lieu
    # df_cible['Lieu_CleLieu']
    # df_cible['Lieu_IdLieu']=
    # df_cible['Lieu_SeqDistinctiveLieu']
    df_cible['Lieu_DtHrChargement1']=CHARGEMENT
    # df_cible['Lieu_CleEvenementSource']
    # df_cible['Lieu_DescLieu']
    df_cible['Lieu_NomLongLieu'] = '{extraction de df_source_auteur}'+df_source['organisateur']
    df_cible['Lieu_EnonceSource'] = df_source['notice_complete']
    # df_cible['Lieu_CoordonneesKMLEpicentre']
    # df_cible['Lieu_PolygoneKMLZone']
    df_cible['Lieu_Analyste'] = "compilé par Julien Vallières et Luc Gauvreau"
    df_cible['Lieu_ReferenceBiblio'] = "{1933-1962 : Programme du [no congrès] congrès, [année du congrès]/1963-2001 : Programme général : [no congrès] congrès annuel, [année du congrès]}"


    #Transfert vers la table Document Source
    #Transfert vers la table Sujet
    df_cible['Sujet_TitreSujet']=df_source['seance_sujet']
    # df_cible['Sujet_CleSujet]
    # df_cible['Sujet_IdSujet]
    # df_cible['Sujet_SeqDistinctiveSujet]
    df_cible['Sujet_DtHrChargement1']=CHARGEMENT
    # df_cible['Sujet_CleEvenementSource]
    df_cible['Sujet_EnonceSource'] = df_source['notice_complete']
    # df_cible['Sujet_TitreLongSujet]
    # df_cible['Sujet_DescSujet]=df_source['']
    df_cible['Sujet_Analyste'] = "compilé par Julien Vallières et Luc Gauvreau"
    df_cible['Sujet_ReferenceBiblio'] = "{1933-1962 : Programme du [no congrès] congrès, [année du congrès]/1963-2001 : Programme général : [no congrès] congrès annuel, [année du congrès]}"

    #Inscrit la date de l'ingestion dans le dataset
    x=datetime.now()
    df_cible['Date_Ingestion']=x

    #display(df_cible)

    return(df_cible)

In [ ]:
column_definition_file_path =  os.path.join(project_dir,"Ingestion_all",r"Table_Enonce_Initialise.xlsx")
source_data_file_path =  os.path.join(project_dir,"Ingestion_all",r"input/programme.csv")
df_cible = pd.read_excel(column_definition_file_path)
DF = process_file(source_data_file_path, df_cible)
#display(DF)

le fichier csv /content/gdrive/My Drive/Colab Data/ACFAS/Ingestion_all/input/programme.csv  sera traité


<ipython-input-25-848abb4f1c0f>:6: DtypeWarning: Columns (0,4,5,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_source = pd.read_csv(file_path)


#Export data

##System information

Note on what type of machine this code was executed.

In [ ]:
info={}
info['time'] = datetime.now()
info['platform'] = platform.system()
info['platform-release'] = platform.release()
info['platform-version'] = platform.version()
info['architecture'] = platform.machine()
info['hostname'] = socket.gethostname()
info['ip-address'] = socket.gethostbyname(socket.gethostname())
info['mac-address'] = ':'.join(re.findall('..', '%012x' % uuid.getnode()))
info['processor'] = platform.processor()
info['threads'] = str(psutil.cpu_count()) + ' logical cores'
try:
  info['speed'] = str(round(psutil.cpu_freq().current)) + " Mhz (currently)"
except:
  info['speed'] = 'unknown'
info['ram'] = str(round(psutil.virtual_memory().total / (1024.0 **3)))+' Go'
info['disk'] = str(round(psutil.disk_usage('/').total / (1024.0 **3)))+' Go'
try:
  info['gpu_total_ram'] = str(round(GPUtil.getGPUs()[0].memoryTotal / 1024.0))+' Go'
except:
  info['gpu_total_ram'] = 'unknown'
try:
  info['gpu_free_ram'] = str(round(GPUtil.getGPUs()[0].memoryFree / 1024.0))+' Go'
except:
  info['gpu_free_ram'] = 'unknown'
try:
  info['gpu_name'] = GPUtil.getGPUs()[0].name
except:
  info['gpu_name'] = 'unknown'
DF_info = pd.DataFrame.from_dict(info, orient='index', columns=['Value'])

##Requirements

Note the packages currently imported into the working environment.

In [ ]:
module_names = set()
for name, val in list(globals().items()):  # Create a copy of items to avoid iteration issues
    if isinstance(val, types.ModuleType):
        module_names.add(val.__name__)

filepath=os.path.join(operation_dir, project_name+'_'+operation_name+'_requirements.txt')
with open(filepath, 'w') as file_:
  for name in module_names:
    try:
      version = pkg_resources.get_distribution(name).version
    except:
      version = ""
    file_.write(f"{name}=={version}\n")

DF_requirements = pd.read_csv(filepath, sep="\=\=| @ ", header=None, engine = 'python')
DF_requirements.columns=['pack.','ver.']

##Export dataframe to excel

Assuming the data is in panda dataframe form, let's export it to excel for easy visualization. Let's add the system information and requirements.

In [ ]:
writer = pd.ExcelWriter(os.path.join(operation_dir, project_name+'_'+operation_name+'.xlsx'))
DF_info.to_excel(writer,'System')
DF_requirements.to_excel(writer,'Module')
DF.to_excel(writer,'Data')
writer.close()

##Export dataframe to pickle

Assuming the data is in panda dataframe form, let's export it with pickle for easy manipulation.

In [ ]:
pd.to_pickle(DF,
             os.path.join(operation_dir, project_name+'_'+operation_name+'.pkl'))